In [1]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [12]:
include("utils.jl")
include("visual_utils.jl")

show_eof_and_pc_modes_of_timeline (generic function with 1 method)

In [5]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools
using Statistics

In [4]:
mon_mean_base_path = "/home/denis/workspace/data/ivt_monthly_mean"
day_mean_base_path = "/home/denis/workspace/data/ivt_daily_mean"

"/home/denis/workspace/data/ivt_daily_mean"

In [ ]:
# monthly_tl_data = filter_by_date(build_timeline_data(mon_mean_base_path, 10, "ssp126", "ssp585"; file_range_selection=:)) do timedata
#     return year(timedata) >= 2081 && year(timedata) <= 2100  
# end

In [3]:
winter_months = [12, 1, 2, 3]

4-element Vector{Int64}:
 12
  1
  2
  3

In [6]:

monthly_tl_data = filter_by_date(build_timeline_data(mon_mean_base_path, 10, "ssp126", "ssp585"; file_range_selection = :)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-16T12:00:00"), DateTime("2015-02-14T21:00:00"), DateTime("2015-03-16T09:00:00"), DateTime("2015-12-16T09:00:00"), DateTime("2016-01-16T09:00:00"), DateTime("2016-02-15T09:00:00"), DateTime("2016-03-16T09:00:00"), DateTime("2016-12-16T09:00:00"), DateTime("2017-01-16T09:00:00"), DateTime("2017-02-14T21:00:00")  …  DateTime("2098-12-16T09:00:

In [ ]:
daily_tl_data = filter_by_date(build_timeline_data(day_mean_base_path, 10, "ssp126", "ssp585"; file_range_selection = :)) do time_element

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

In [ ]:
# truth_data = filter_by_date(build_timeline_data(base_path, 10, "ssp126", "ssp585"; file_range_selection=1:2)) do timedata
#     return year(timedata) >= 2015 && year(timedata) <= 2034
# end

In [ ]:
compare_truth_with_tldata(truth_data, tl_data, "mon_mean_comparison.mp4"; framerate = 3, resolution = (2000, 1000), colormap = :managua)

In [8]:
all_scopes = get_sliding_time_scopes_by_threshold(monthly_tl_data.time, 30)

57-element Vector{UnitRange{Int64}}:
 1:120
 4:124
 8:128
 12:132
 16:136
 20:140
 24:144
 28:148
 32:152
 36:156
 ⋮
 194:314
 198:318
 202:322
 206:327
 210:331
 214:335
 218:339
 222:343
 226:347

In [13]:
eof_data = calculate_eofs_of_tl_data(
    monthly_tl_data, 
    all_scopes, 
    2; 
    engine=:python, 
    reof=false, 
    center=true, 
    align_with_mean=true, 
    weights=sqrt.(cos.(deg2rad.(monthly_tl_data.lats))), 
    eof_type=:normal,
    scale_with_eigenvals=true
)

Handled scope 1:120 out of 57 on thread 1
Handled scope 4:124 out of 57 on thread 1
Handled scope 8:128 out of 57 on thread 1
Handled scope 12:132 out of 57 on thread 1
Handled scope 16:136 out of 57 on thread 1
Handled scope 20:140 out of 57 on thread 1
Handled scope 24:144 out of 57 on thread 1
Handled scope 28:148 out of 57 on thread 1
Handled scope 32:152 out of 57 on thread 1
Handled scope 36:156 out of 57 on thread 1
Handled scope 40:160 out of 57 on thread 1
Handled scope 44:165 out of 57 on thread 1
Handled scope 48:169 out of 57 on thread 1
Handled scope 52:173 out of 57 on thread 1
Handled scope 56:177 out of 57 on thread 1
Handled scope 60:181 out of 57 on thread 1
Handled scope 64:185 out of 57 on thread 1
Handled scope 68:189 out of 57 on thread 1
Handled scope 72:193 out of 57 on thread 1
Handled scope 76:197 out of 57 on thread 1
Handled scope 80:201 out of 57 on thread 1
Handled scope 85:205 out of 57 on thread 1
Handled scope 89:209 out of 57 on thread 1
Handled scope 

Dict{String, Vector{EOFResult}} with 2 entries:
  "ssp126" => [EOFResult([-10.4221 -4.85788 … 0.362275 0.285441; -9.86068 -2.86…
  "ssp585" => [EOFResult([-5.32978 1.32659 … 1.07803 0.827362; -5.25707 3.59012…

In [30]:
show_eof_modes_of_timeline(
    monthly_tl_data,
    eof_data,
    all_scopes,
    "eof_6modes_monthly_30seasons_aligned_tst.mp4";
    framerate = 2,
    colormap = :broc,
    coastline_color = :black,
    resolution = (2000, 1100),
    whole_figure_title = "Leading EOF Modes of Daily Averages of IVT (18 Winter Period) Aligned"
)

Threads available: 8


Temporal plot first 2 modes
Compare to mean
scale with singular value (diss)
0 contour plot

In [14]:
show_eof_and_pc_modes_of_timeline(
    monthly_tl_data,
    eof_data,
    all_scopes,
    "eof_2modes_pc_mean_monthly_30seasons_aligned.mp4";
    framerate = 2,
    colormap = :BrBG_3,
    coastline_color = :black,
    resolution = (2000, 2000),
    whole_figure_title = "Leading EOF Modes of Monthly Averages of IVT (30 Winter Period) Aligned"
)

Data extremas: -90.20836332588088 131.42846550342423
Chunk extremas: (10.487220230524036, 373.69820554315754)
Chunk extremas: (10.80574334361819, 363.9009116704155)
Got an update: 1
ssp126
Length time 1: 120
Length temporal mode 1: 120
Length time 2: 120
Length temporal mode 2: 120
ssp585
Length time 1: 120
Length temporal mode 1: 120
Length time 2: 120
Length temporal mode 2: 120
Chunk extremas: (10.456006607262918, 374.50774099741454)
Chunk extremas: (10.727503652636182, 363.71133877989865)
Got an update: 2
ssp126
Length time 1: 121
Length temporal mode 1: 121
Length time 2: 121
Length temporal mode 2: 121
ssp585
Length time 1: 121
Length temporal mode 1: 121
Length time 2: 121
Length temporal mode 2: 121
Chunk extremas: (10.452622487165216, 374.09845120533225)
Chunk extremas: (10.664949944033621, 366.49350100397095)
Got an update: 3
ssp126
Length time 1: 121
Length temporal mode 1: 121
Length time 2: 121
Length temporal mode 2: 121
ssp585
Length time 1: 121
Length temporal mode 1: 1